In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn import preprocessing
import sklearn.metrics as sm
from sklearn import svm
import re
from sklearn.model_selection import train_test_split
import os
import subprocess
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier

### 1. Clean each datasets

#### 1. Observe each datasets

In [108]:
# read in the datasets as string

tags = pd.read_csv('dataset/tags.csv',encoding = "utf-8",dtype = str) 
movies = pd.read_csv('dataset/movies.csv',encoding = "utf-8",dtype = str)
ratings = pd.read_csv('dataset/ratings.csv',encoding = "utf-8",dtype = str)
gs = pd.read_csv('dataset/genome-scores.csv',encoding = "utf-8",dtype = str)
gt = pd.read_csv('dataset/genome-tags.csv',encoding = "utf-8",dtype = str)
links = pd.read_csv('dataset/links.csv',encoding = "utf-8",dtype = str) 

In [4]:
# observe information of each datasets

In [5]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465564 entries, 0 to 465563
Data columns (total 4 columns):
userId       465564 non-null object
movieId      465564 non-null object
tag          465548 non-null object
timestamp    465564 non-null object
dtypes: object(4)
memory usage: 14.2+ MB


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
movieId    27278 non-null object
title      27278 non-null object
genres     27278 non-null object
dtypes: object(3)
memory usage: 639.4+ KB


In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
userId       object
movieId      object
rating       object
timestamp    object
dtypes: object(4)
memory usage: 610.4+ MB


In [8]:
gs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
movieId      object
tagId        object
relevance    object
dtypes: object(3)
memory usage: 268.0+ MB


In [9]:
gt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
tagId    1128 non-null object
tag      1128 non-null object
dtypes: object(2)
memory usage: 17.7+ KB


In [10]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
movieId    27278 non-null object
imdbId     27278 non-null object
tmdbId     27026 non-null object
dtypes: object(3)
memory usage: 639.4+ KB


In [11]:
# check for missing data

In [109]:
tags.isnull().sum().sort_values(ascending=False)

tag          16
timestamp     0
movieId       0
userId        0
dtype: int64

In [110]:
movies.isnull().sum().sort_values(ascending=False)

genres     0
title      0
movieId    0
dtype: int64

In [111]:
ratings.isnull().sum().sort_values(ascending=False)

timestamp    0
rating       0
movieId      0
userId       0
dtype: int64

In [112]:
gs.isnull().sum().sort_values(ascending=False)

relevance    0
tagId        0
movieId      0
dtype: int64

In [113]:
gt.isnull().sum().sort_values(ascending=False)

tag      0
tagId    0
dtype: int64

In [114]:
links.isnull().sum().sort_values(ascending=False)

tmdbId     252
imdbId       0
movieId      0
dtype: int64

In [18]:
# Extract some data for observation

In [19]:
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [20]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [22]:
gs.head()

,movieId,tagId,relevance
0,1,1,0.025000000000000022
1,1,2,0.025000000000000022
2,1,3,0.057750000000000024
3,1,4,0.09675
4,1,5,0.14675


In [23]:
gt.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [24]:
links.head()

,movieId,imdbId,tmdbId
0,1,0114709,862
1,2,0113497,8844
2,3,0113228,15602
3,4,0114885,31357
4,5,0113041,11862


In [115]:
# Movie titles are entered manually or imported from <https://www.themoviedb.org/>.
np.set_printoptions(threshold=np.inf)
movies['title'].unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       'Waiting to Exhale (1995)', 'Father of the Bride Part II (1995)',
       'Heat (1995)', 'Sabrina (1995)', 'Tom and Huck (1995)',
       'Sudden Death (1995)', 'GoldenEye (1995)',
       'American President, The (1995)',
       'Dracula: Dead and Loving It (1995)', 'Balto (1995)',
       'Nixon (1995)', 'Cutthroat Island (1995)', 'Casino (1995)',
       'Sense and Sensibility (1995)', 'Four Rooms (1995)',
       'Ace Ventura: When Nature Calls (1995)', 'Money Train (1995)',
       'Get Shorty (1995)', 'Copycat (1995)', 'Assassins (1995)',
       'Powder (1995)', 'Leaving Las Vegas (1995)', 'Othello (1995)',
       'Now and Then (1995)', 'Persuasion (1995)',
       'City of Lost Children, The (Cité des enfants perdus, La) (1995)',
       'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
       'Dangerous Minds (1995)',
       'Twelve Monkeys (a.k.a. 12 Monkeys) (1995)',
       'Wings of Courage (1995)', 

In [174]:
# Errors and inconsistencies exist in these titles. 

#### 2. Adjust data types

In [116]:
tags['timestamp'] = tags['timestamp'].astype('int')
ratings['rating'] = ratings['rating'].astype('float')
ratings['timestamp'] = ratings['timestamp'].astype('int')
gs['relevance'] = gs['relevance'].astype('float')
tags['movieId'] = tags['movieId'].astype('int')
movies['movieId'] = movies['movieId'].astype('int')
ratings['movieId'] = ratings['movieId'].astype('int')
gs['movieId'] = gs['movieId'].astype('int')
links['movieId'] = links['movieId'].astype('int')

In [27]:
tags.dtypes

userId       object
movieId       int64
tag          object
timestamp     int64
dtype: object

In [28]:
ratings.dtypes

userId        object
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [29]:
gs.dtypes

movieId        int64
tagId         object
relevance    float64
dtype: object

#### 3. Check for logic errors

In [117]:
# relevance and rating cannot be negative
querySer=ratings.loc[:,'rating']>0
print('ratings-before：',ratings.shape)
ratings=ratings.loc[querySer,:]
print('ratings-after：',ratings.shape)

querySer=gs.loc[:,'relevance']>0
print('gs-before：',gs.shape)
gs=gs.loc[querySer,:]
print('gs-after：',gs.shape)

ratings-before： (20000263, 4)
ratings-after： (20000263, 4)
gs-before： (11709768, 3)
gs-after： (11709768, 3)


In [93]:
# Logic errors don't exist in this dataset.

#### 4. Uniform format

In [118]:
# movies title
movies['title']= movies['title'].str.strip()
movies['title']= movies['title'].str.title()
movies['title'].unique()

array(['Toy Story (1995)', 'Jumanji (1995)', 'Grumpier Old Men (1995)',
       'Waiting To Exhale (1995)', 'Father Of The Bride Part Ii (1995)',
       'Heat (1995)', 'Sabrina (1995)', 'Tom And Huck (1995)',
       'Sudden Death (1995)', 'Goldeneye (1995)',
       'American President, The (1995)',
       'Dracula: Dead And Loving It (1995)', 'Balto (1995)',
       'Nixon (1995)', 'Cutthroat Island (1995)', 'Casino (1995)',
       'Sense And Sensibility (1995)', 'Four Rooms (1995)',
       'Ace Ventura: When Nature Calls (1995)', 'Money Train (1995)',
       'Get Shorty (1995)', 'Copycat (1995)', 'Assassins (1995)',
       'Powder (1995)', 'Leaving Las Vegas (1995)', 'Othello (1995)',
       'Now And Then (1995)', 'Persuasion (1995)',
       'City Of Lost Children, The (Cité Des Enfants Perdus, La) (1995)',
       'Shanghai Triad (Yao A Yao Yao Dao Waipo Qiao) (1995)',
       'Dangerous Minds (1995)',
       'Twelve Monkeys (A.K.A. 12 Monkeys) (1995)',
       'Wings Of Courage (1995)', 

#### 5. Remove missing data

In [119]:
tags = tags.isnull().dropna()
tags.isnull().sum().sort_values(ascending=False)

timestamp    0
tag          0
movieId      0
userId       0
dtype: int64

In [120]:
links = links.isnull().dropna()
links.isnull().sum().sort_values(ascending=False)

tmdbId     0
imdbId     0
movieId    0
dtype: int64

#### 6. Encoding movie genres and title

In [121]:
movies_title = ['movieId', 'title', 'genres']
movies_orig = movies.values
#drop the year in Title
pattern = re.compile(r'^(.*)\((\d+)\)$')
    
title_map = {  }
for ii,val in enumerate(set(movies['title'])):
    if pattern.match(val)!=None:
        title_map[val]=pattern.match(val).group(1)
    else:
        title_map[val]=val
movies['title'] = movies['title'].map(title_map)

#encoding genres
genres_set = set()
for val in movies['genres'].str.split('|'):
    genres_set.update(val)

genres_set.add('<PAD>')
genres2int = {val:ii for ii, val in enumerate(genres_set)}

#change genres to 18d digital list
genres_map = {val:[genres2int[row] for row in val.split('|')] for ii,val in enumerate(set(movies['genres']))}

for key in genres_map:
    for cnt in range(max(genres2int.values()) - len(genres_map[key])):
        genres_map[key].insert(len(genres_map[key]) + cnt,genres2int['<PAD>'])
    
movies['genres'] = movies['genres'].map(genres_map)

#encoding title
title_set = set()
for iter_i,val in enumerate(movies['title'].str.split()):
    title_set.update(val)
    
title_set.add('<PAD>')
title2int = {val:ii for ii, val in enumerate(title_set)}

#change title to 15d digital list
title_count = 15
title_map = {val:[title2int[row] for row in val.split()] for ii,val in enumerate(set(movies['title']))}
    
for key in title_map:
    for cnt in range(title_count - len(title_map[key])):
        title_map[key].insert(len(title_map[key]) + cnt,title2int['<PAD>'])
    
movies['title'] = movies['title'].map(title_map)

movies.head()

,movieId,title,genres
0,1,"[5686, 19233, 14708, 14708, 14708, 14708, 1470...","[8, 16, 0, 3, 12, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5..."
1,2,"[11463, 14708, 14708, 14708, 14708, 14708, 147...","[8, 0, 12, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,..."
2,3,"[17866, 10837, 24412, 14708, 14708, 14708, 147...","[3, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
3,4,"[24779, 13285, 15242, 14708, 14708, 14708, 147...","[3, 9, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
4,5,"[27523, 795, 20110, 21850, 6674, 5546, 14708, ...","[3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."


#### 7. Getting the average ratings for each movie

In [105]:
# Extract data of rating and movieid
average_rating = ratings[['movieId','rating']]

# calculate the average rating for each movie
mean_rating = round(average_rating.groupby('movieId').mean()*100,0)
mean_rating.head()

,rating
movieId,
1,392.0
2,321.0
3,315.0
4,286.0
5,306.0


In [122]:
import statistics
a=[]
b=[]
for x in movies['title']:
    a.append(round(statistics.mean(x),0))
for x in movies['genres']:
    b.append(round(statistics.mean(x),0))

#### 8. Join all useful data to one dataframe

In [127]:
i=0;
j=0;
for aa in a:
    movies['title'][i]=aa
    i+=1;
    if i==50:
        break;
for bb in b:
    movies['genres'][j]=bb
    j+=1;
    if j==50:
        break;

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [154]:
# Change list of tags/movies/average_ratings/gs/gt/links to dataframe
tagsDF = pd.DataFrame(tags)
mean_ratingDF = pd.DataFrame(mean_rating)
moviesDF = pd.DataFrame(movies)
gsDF = pd.DataFrame(gs)
gtDF = pd.DataFrame(gt)
linksDF = pd.DataFrame(links)

# Join them together
Jointable = mean_ratingDF.join(moviesDF.set_index('movieId'), on='movieId')
Jointable = Jointable.join(gsDF.set_index('movieId'), on='movieId')
#Jointable = Jointable.join(tagsDF.set_index('movieId'), on='movieId')
#Jointable = Jointable.join(linksDF.set_index('movieId'), on='movieId')

Jointable.head()

,rating,title,genres,tagId,relevance
movieId,,,,,
1,392.0,14408,6,1,25.0
1,392.0,14408,6,2,25.0
1,392.0,14408,6,3,58.0
1,392.0,14408,6,4,97.0
1,392.0,14408,6,5,147.0


In [153]:
gs['relevance']=round(gs['relevance']*1000,0)

#### 9. Split dataset to two parts

In [138]:
def load_data():
    target_fields = 'rating'
    features_pd, targets_pd = Jointable.drop(target_fields, axis=1), Jointable[target_fields]
    
    features = features_pd.values
    targets_values = targets_pd.values
    return features,targets_values 

### 2. Movie-rating prediction using SVM

#### 1. Predicting function

In [63]:
def svm_fit_predict(X_train, X_test ,y_train):
    
    clf = svm.SVC(C=2,kernel='rbf',gamma=0.001, class_weight='balanced')
    _X_train=X_train.reshape(X_train.shape[0], -1)
    _X_train=np.nan_to_num(_X_train)
    clf.fit(np.array(_X_train), np.array(y_train))
    test_data = X_test.reshape(X_test.shape[0], -1)
    y_test_predict = clf.predict(test_data)
    return y_test_predict

#### 2.Computing accuracy

In [41]:
def compute_Pr_Re(y_test_pre,y_test):
    assert len(y_test_pre)==len(y_test), 'len(outputs) != len(labels)'
    print('mean absolute error=',round(sm.mean_absolute_error(y_test,y_test_pre),2))
    print('mean squared error=',round(sm.mean_squared_error(y_test,y_test_pre),2))
    print('median absolute error=',round(sm.median_absolute_error(y_test,y_test_pre),2))
    print('explained variance score=',round(sm.explained_variance_score(y_test,y_test_pre),2))
    print('R2 score=',round(sm.r2_score(y_test,y_test_pre),2))

#### 3.Training this model

In [187]:
X_features, labels=load_data()

In [188]:
X_features=np.nan_to_num(X_features[0:50000])
y_train=np.nan_to_num(y_train[0:50000])

In [189]:
labels=labels[0:50000]

In [194]:
X_train, X_test, y_train, y_test = train_test_split(X_features, labels, test_size=0.2, random_state=52)
clf = svm.SVC(C=0.8, kernel='rbf', gamma=20, decision_function_shape='ovr')

In [182]:
clf.fit(X_train, y_train)
y_test_predict = clf.predict(X_test)
compute_Pr_Re(y_test_predict,y_test)

mean absolute error= 41.68
mean squared error= 2526.37
median absolute error= 37.0
explained variance score= 0.0
R2 score= -0.61


In [179]:
np.unique(y_train)

array([261., 262., 265., 272., 286., 288., 300., 301., 302., 306., 314.,
       315., 320., 321., 325., 327., 332., 337., 338., 341., 343., 347.,
       350., 358., 363., 364., 365., 367., 369., 379., 383., 385., 390.,
       392., 394., 395., 397., 406.])

### 3. Movie-rating prediction using KNN

#### 1. Predicting function

In [52]:
def svm_fit_predict(X_train, X_test ,y_train):
    
    clf = KNeighborsClassifier(n_neighbors=2)
    _X_train=X_train.reshape(X_train.shape[0], -1)
    clf.fit(_X_train, y_train)
    test_data = X_test.reshape(X_test.shape[0], -1)
    y_test_predict = clf.predict(test_data)
    return y_test_predict

#### 2.Computing accuracy

In [53]:
def compute_Pr_Re(y_test_pre,y_test):
    assert len(y_test_pre)==len(y_test), 'len(outputs) != len(labels)'
    print('mean absolute error=',round(sm.mean_absolute_error(y_test,y_test_pre),2))
    print('mean squared error=',round(sm.mean_squared_error(y_test,y_test_pre),2))
    print('median absolute error=',round(sm.median_absolute_error(y_test,y_test_pre),2))
    print('explained variance score=',round(sm.explained_variance_score(y_test,y_test_pre),2))
    print('R2 score=',round(sm.r2_score(y_test,y_test_pre),2))

#### 3.Training this model

In [185]:
X_train, X_test, y_train, y_test = train_test_split(X_features, labels, test_size=0.2, random_state=52)
clf = KNeighborsClassifier(n_neighbors=2)
clf.fit(X_train, y_train)
y_test_pre=clf.predict(X_test)
compute_Pr_Re(y_test_pre,y_test)

mean absolute error= 4.01
mean squared error= 195.98
median absolute error= 0.0
explained variance score= 0.88
R2 score= 0.87


### 4. Movie-rating prediction using DecisionTree 

#### 1. Predicting function

In [49]:
def svm_fit_predict(X_train, X_test ,y_train):
    
    clf = DecisionTreeClassifier()
    _X_train=X_train.reshape(X_train.shape[0], -1)
    clf.fit(_X_train, y_train)
    test_data = X_test.reshape(X_test.shape[0], -1)
    y_test_predict = clf.predict(test_data)
    return y_test_predict

#### 2.Computing accuracy

In [50]:
def compute_Pr_Re(y_test_pre,y_test):
    assert len(y_test_pre)==len(y_test), 'len(outputs) != len(labels)'
    print('mean absolute error=',round(sm.mean_absolute_error(y_test,y_test_pre),2))
    print('mean squared error=',round(sm.mean_squared_error(y_test,y_test_pre),2))
    print('median absolute error=',round(sm.median_absolute_error(y_test,y_test_pre),2))
    print('explained variance score=',round(sm.explained_variance_score(y_test,y_test_pre),2))
    print('R2 score=',round(sm.r2_score(y_test,y_test_pre),2))

#### 3.Training this model

In [186]:
X_train, X_test, y_train, y_test = train_test_split(X_features, labels, test_size=0.2, random_state=52)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_test_pre=clf.predict(X_test)
compute_Pr_Re(y_test_pre,y_test)

mean absolute error= 0.0
mean squared error= 0.0
median absolute error= 0.0
explained variance score= 1.0
R2 score= 1.0


----------

### 3. Getting additional data from IMDb's Web server using imdbId

In [33]:
pip install git+https://github.com/alberanid/imdbpy

  Cloning https://github.com/alberanid/imdbpy to /private/var/folders/wg/rn1gx30s3j7_74yqng8tcbmr0000gn/T/pip-req-build-chb8es8a
  Running command git clone -q https://github.com/alberanid/imdbpy /private/var/folders/wg/rn1gx30s3j7_74yqng8tcbmr0000gn/T/pip-req-build-chb8es8a
  Stored in directory: /private/var/folders/wg/rn1gx30s3j7_74yqng8tcbmr0000gn/T/pip-ephem-wheel-cache-m5nrym2b/wheels/0f/09/61/190df5e0276765680540f1562f2abca80e725a7e48595e993f
Successfully built IMDbPY
  Found existing installation: IMDbPY 6.9.dev20191205
    Uninstalling IMDbPY-6.9.dev20191205:
      Successfully uninstalled IMDbPY-6.9.dev20191205
Note: you may need to restart the kernel to use updated packages.


In [34]:
links.shape

(27278, 3)

In [35]:
from imdb import IMDb

# create an instance of the IMDb class
ia = IMDb()
n = 0
movieinfo = []
directors = []

# get movies
while n < 1:
    movie = ia.get_movie(links['imdbId'][n])
   #movieinfo.append(movie)
    print(movie)
    n+=1


2019-12-06 09:23:56,969 CRITICAL [imdbpy] //anaconda3/lib/python3.7/site-packages/imdb/_exceptions.py:34: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0000000/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': <HTTPError 404: 'Not Found'>},); kwds: {}
Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/site-packages/imdb/parser/http/__init__.py", line 219, in retrieve_unicode
    response = uopener.open(url)
  File "//anaconda3/lib/python3.7/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "//anaconda3/lib/python3.7/urllib/request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "//anaconda3/lib/python3.7/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "//anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "//anaconda3/lib/p